In [ ]:
### XGBOOST with non-parametric confidence intervals

import pandas as pd
import numpy as np
import snowflake.connector
import xgboost as xgb
import pickle
import matplotlib.pyplot as plt

In [ ]:
## Select predictors. We select without UMC_30.

predictors = ["AGE", "GENDER", 
                "UMC_HEART",                 "UMC_DEMENTIA" , 
                "UMC_CKD",                   "UMC_INHERITED_METABOLIC",   "UMC_HYP",
                "UMC_DM2",                   "UMC_OBESITY",               "UMC_CBD",
                "UMC_COPD",                  "UMC_HYL",                   "UMC_ASTHMA",
                "UMC_CANCER",                "UMC_DM1",                   "UMC_LIVER",
                "UMC_PREGNANCY",             "UMC_PULM_FIB",              "UMC_RHEUMATOID_ARTHRITIS",
                "UMC_PARKINSONS",            "UMC_PANCREATITIS",         "UMC_DEV_BEH_DISORDER",
                "UMC_PROSTATE_CANCER",       "UMC_LUNG_CANCER",           "UMC_COLORECTAL_CANCER", 
                "UMC_BREAST_CANCER",         "UMC_IMMUNE_DEF",            "UMC_LYMPHOMA_MYELOMA",
                "UMC_LUPUS",                 "UMC_MULTIPLE_SCLEROSIS",    "UMC_IMMUNE_SUPPRESSANTS",
                "UMC_TRANS", "WEEKS_SINCE_2020"]
predictors

In [ ]:
# Define hyperparameter tuning function

def xgboost_hypertuning(training_set):
    
    # Note: This function follows Kevin Lemagnen's guide (published on the CambridgeSpark blog) closely. Thanks Kevin!
    
    import time
    current_time = time.strftime('%H:%M:%S', time.localtime())
    print(current_time)
    # Select parameters to tune
    params = {
        'max_depth':6,
        'min_child_weight': 1,
        'eta': .1, 
        'subsample': 1, 
        'colsample_bytree': 1,
        'objective':'binary:logistic',
        'gamma': 0
    }

    # Add eval metric to params
    params['eval_metric'] = "logloss"

    # Set max boosting rounds
    num_boost_round = 3000

    # Tune max_depth and min_child_weight
    gridsearch_params = [
        (max_depth, min_child_weight)
        for max_depth in range(2, 6)
        for min_child_weight in range(4,15)
    ]

    min_ll = float("Inf")
    best_params = None
    for max_depth, min_child_weight in gridsearch_params:
        print("CV with max_depth={}, min_child_weight={}".format(max_depth, min_child_weight))

        # Update parameters
        params['max_depth'] = max_depth
        params['min_child_weight'] = min_child_weight

        # Run CV
        cv_results = xgb.cv(
        params,
        training_set,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'logloss'},
        early_stopping_rounds=10)

        mean_ll = cv_results['test-logloss-mean'].min()
        boost_rounds = cv_results['test-logloss-mean'].argmin()
        print("\tLogloss {} for {} rounds".format(mean_ll, boost_rounds))
        if mean_ll < min_ll:
            min_ll = mean_ll
            best_params = (max_depth, min_child_weight)

    print("Best params: {}, {}, Logloss: {}".format(best_params[0], best_params[1], min_ll))

    current_time = time.strftime('%H:%M:%S', time.localtime())
    print(current_time)
    # Set these and continue tuning:
    params['max_depth'] = best_params[0]
    params['min_child_weight'] = best_params[1]

    # Tune subsample and colsample_bytree
    gridsearch_params = [
        (subsample, colsample)
        for subsample in [i/10. for i in range(6,11)]
        for colsample in [i/10. for i in range(6,11)]
    ] 

    min_ll = float("Inf")
    best_params = None
    for subsample, colsample in reversed(gridsearch_params):
        print("CV with subsample={}, colsample={}".format(subsample, colsample))

        # Update parameters
        params['subsample'] = subsample
        params['colsample_bytree'] = colsample

        # Run CV
        cv_results = xgb.cv(
            params,
            training_set,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics={'logloss'},
            early_stopping_rounds=10)

        mean_ll = cv_results['test-logloss-mean'].min()
        boost_rounds = cv_results['test-logloss-mean'].argmin()
        print("\tLogloss {} for {} rounds".format(mean_ll, boost_rounds))
        if mean_ll < min_ll:
            min_ll = mean_ll
            best_params = (subsample, colsample)

    print("Best params: {}, {}, Logloss: {}".format(best_params[0], best_params[1], min_ll))

    current_time = time.strftime('%H:%M:%S', time.localtime())
    print(current_time)
    # Set these and continue tuning
    params['subsample'] = best_params[0]
    params['colsample_bytree'] = best_params[0]

    min_ll = float("Inf")
    best_params = None
    for gamma in [0, .5, 1, 5]:
        print("CV with gamma={}".format(gamma))

        # Update parameters
        params['gamma'] = gamma

        # Run CV
        cv_results = xgb.cv(
            params,
            training_set,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics={'logloss'},
            early_stopping_rounds=10)

        mean_ll = cv_results['test-logloss-mean'].min()
        boost_rounds = cv_results['test-logloss-mean'].argmin()
        print("\tLogloss {} for {} rounds".format(mean_ll, boost_rounds))
        if mean_ll < min_ll:
            min_ll = mean_ll
            best_params = gamma

    print("Best params: {}, Logloss: {}".format(best_params, min_ll))
    
    current_time = time.strftime('%H:%M:%S', time.localtime())
    print(current_time)
    
    params['gamma'] = best_params

    %time

    # Tune ETA
    min_ll = float("Inf")
    best_params = None

    for eta in [.2, .1, .05, .01]:
        print("CV with eta = {}".format(eta))

        # Update parameter
        params['eta'] = eta

        # Run and time CV
        %time 
        cv_results = xgb.cv(
            params,
            training_set,
            num_boost_round = num_boost_round,
            seed=42,
            nfold=5,
            metrics=['logloss'],
            early_stopping_rounds=10)

        # Update best score
        mean_ll = cv_results['test-logloss-mean'].min()
        boost_rounds = cv_results['test-logloss-mean'].argmin()
        print("\tLogloss {} for {} rounds\n".format(mean_ll, boost_rounds))
        if mean_ll < min_ll:
            min_ll = mean_ll
            best_params = eta

    print("Best params: {}, Logloss: {}".format(best_params, min_ll))

    params['eta'] = best_params
    current_time = time.strftime('%H:%M:%S', time.localtime())
    print(current_time)

    # Test on optimal set
    #model = xgb.train(params, 
    #                  training_set,
    #                  num_boost_round = num_boost_round,
    #                  evals = [(dtest, "Test")],
    #                  early_stopping_rounds = 10)

    #print("Best Logloss: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))
    print(params)
    current_time = time.strftime('%H:%M:%S', time.localtime())
    print(current_time)
    return params

In [ ]:
## Hyperparameter tuning on Deaths dataset:

In [ ]:
# Load augmented  dataset:
mdf = pd.read_csv('DEBIASED_DATA.csv')

# Select date-range:
mdf = mdf[(mdf['DAYS_SINCE_2020'] >= 121) & (mdf['DAYS_SINCE_2020'] <= 335)]

mdf.describe()

In [ ]:
# Make model matrix
X = mdf
Y = X["DIED"]

X = X[predictors]

# Tuning on 90% model:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .10, random_state=42)

dtrain = xgb.DMatrix(X_train, label = y_train)
dtest = xgb.DMatrix(X_test, label = y_test)

In [ ]:
# Tune hyperparameters:
death_params = xgboost_hypertuning(dtrain)

In [ ]:
# Find optimal nrounds based on 90% validation set:
model = xgb.train(death_params, 
                  dtrain,
                  num_boost_round = 9999,
                  evals = [(dtest, "Test")],
                  early_stopping_rounds = 10)

print("Best Logloss: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

In [ ]:
# Load augmented  dataset:
mdf = pd.read_csv('DEBIASED_DATA_HOSPITALIZATIONS.csv')

# Select date-range:
mdf = mdf[(mdf['DAYS_SINCE_2020'] >= 121) & (mdf['DAYS_SINCE_2020'] <= 335)]

mdf.describe()

In [ ]:
# Make model matrix
X = mdf
Y = X["HOSPITALIZED"]

X = X[predictors]

# Tuning on 90% model:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .10, random_state=42)

dtrain = xgb.DMatrix(X_train, label = y_train)
dtest = xgb.DMatrix(X_test, label = y_test)

In [ ]:
# Tune hyperparameters:
hosp_params = xgboost_hypertuning(dtrain)

In [ ]:
# Find optimal nrounds based on 90% validation set:
model = xgb.train(hosp_params, 
                  dtrain,
                  num_boost_round = 9999,
                  evals = [(dtest, "Test")],
                  early_stopping_rounds = 10)

print("Best Logloss: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

In [ ]:
# We next save the model estimate, and 100 bootstrap models (deaths)

In [ ]:
# Load augmented  dataset:
mdf = pd.read_csv('DEBIASED_DATA.csv')

# Select date-range:
mdf = mdf[(mdf['DAYS_SINCE_2020'] >= 121) & (mdf['DAYS_SINCE_2020'] <= 335)]

# Make model matrix
X = mdf
Y = X["DIED"]
X = X[predictors]

# Fit with tuned hyperparameters:
xgb_fit = xgb.XGBClassifier('binary:logistic', n_estimators = 227,
                            max_depth = 3,
                            min_child_weight = 12,
                            eta = 0.1,
                            subsample = 0.9,
                            colsample_bytree = 0.9,
                            gamma = 1,
                            eval_metric = 'logloss'
                           )

xgb_fit = xgb_fit.fit(X, Y)

# Save model
file_name = "XGB_model_estimate.pkl"
pickle.dump(xgb_fit, open(file_name, "wb"))

print('Model estimate fitted and saved.')

# Make confidence intervals by making n models and saving them
for i in range(1, 101):
    X = mdf.sample(n=len(mdf), replace = True)
    Y = X["DIED"]
    X = X[predictors]
    xgb_fit = xgb.XGBClassifier('binary:logistic', n_estimators = 227,
                            max_depth = 3,
                            min_child_weight = 12,
                            eta = 0.1,
                            subsample = 0.9,
                            colsample_bytree = 0.9,
                            gamma = 1,
                            eval_metric = 'logloss'
                           )
    xgb_fit_ci = xgb_fit.fit(X, Y )
    file_name = "XGB_model_ci_" + str(i) + ".pkl"
    pickle.dump(xgb_fit_ci, open(file_name, "wb"))
    print(i)

In [ ]:
# We next save the model estimate, and 100 bootstrap models (hospitalizations)
# Load augmented  dataset:
mdf = pd.read_csv('DEBIASED_DATA_HOSPITALIZATIONS.csv')

# Select date-range:
mdf = mdf[(mdf['DAYS_SINCE_2020'] >= 121) & (mdf['DAYS_SINCE_2020'] <= 335)]

# Make model matrix
X = mdf
Y = X["HOSPITALIZED"]
X = X[predictors]

# Fit with tuned hyperparameters:
xgb_fit = xgb.XGBClassifier('binary:logistic', n_estimators = 526,
                            max_depth = 5,
                            min_child_weight = 10,
                            eta = 0.05,
                            subsample = 0.8,
                            colsample_bytree = 0.8,
                            gamma = 5,
                            eval_metric = 'logloss'
                           )

xgb_fit = xgb_fit.fit(X, Y)

# Save model
file_name = "XGB_model_hospitalization_estimate.pkl"
pickle.dump(xgb_fit, open(file_name, "wb"))

print('Model estimate fitted and saved.')

for i in range(1, 101):
    X = mdf.sample(n=len(mdf), replace = True)
    Y = X["HOSPITALIZED"]
    X = X[predictors]
    xgb_fit_ci = xgb.XGBClassifier('binary:logistic', n_estimators = 526,
                            max_depth = 5,
                            min_child_weight = 10,
                            eta = 0.05,
                            subsample = 0.8,
                            colsample_bytree = 0.8,
                            gamma = 5,
                            eval_metric = 'logloss'
                           )
    xgb_fit_ci = xgb_fit.fit(X, Y )
    file_name = "XGB_model_hospitalization_ci_" + str(i) + ".pkl"
    pickle.dump(xgb_fit_ci, open(file_name, "wb"))
    print(i)



In [ ]:
# We then make predictions for a random sample from the 103 million, for both hospitalizations and deaths, and export a csv:
rep_sample = pd.read_csv(r'representative_sample_from_full_data.csv')
rep_sample['WEEKS_SINCE_2020'] = 48

# Load death model:
death_model = pickle.load(open('XGB_model_estimate.pkl', 'rb'))
rep_sample['DEATH_PREDICTION'] = pd.DataFrame(death_model.predict_proba(rep_sample[predictors])[:, 1])

# Load hospitalization model:
hosp_model = pickle.load(open('XGB_model_hospitalization_estimate.pkl', 'rb'))
rep_sample['HOSPITALIZATION_PREDICTION'] = pd.DataFrame(hosp_model.predict_proba(rep_sample[predictors])[:, 1])

# Save:
rep_sample = rep_sample[['AGE', 'GENDER', "UMC_30", 'DEATH_PREDICTION', 'HOSPITALIZATION_PREDICTION']]
rep_sample.to_csv(r'representative_sample_narrow.csv')
rep_sample.describe()
